In [ ]:
!pip install pytest-filter-subpackage>=0.1
!pip install pytest-cov>=2.0
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install torch torchvision --upgrade

In [ ]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
mydataset = load_dataset('csv', data_files={'train': 'data/training_for_HF.csv', 'validation': 'data/validation_for_HF.csv',
                                              'test': 'data/test_for_HF.csv'})

In [ ]:
mydataset

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
metric = load_metric('glue', actual_task)

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer2 = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True)


In [ ]:
preprocess_function(mydataset['train'][:5])

In [ ]:
encoded_dataset = mydataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 13
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels
)


In [ ]:
metric_name = "matthews_correlation" # "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
import numpy as np
def to_int(x):
    return int(x)

vfunc = np.vectorize(to_int)

def get_accuracy(preds, labels):
    correct = (preds==labels)
    correct2 = vfunc(correct)
    return sum(correct2)/len(correct2)


In [ ]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = get_accuracy(predictions, labels)
    rez = metric.compute(predictions=predictions, references=labels)
    rez['accuracy'] = acc
    return rez

In [ ]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(eval_dataset=encoded_dataset["test"])